In [1]:
import os
from dotenv import load_dotenv
from google.genai import types
from google.adk.agents import Agent, ParallelAgent, SequentialAgent
from google.adk.models.google_llm import Gemini
from pathlib import Path
from agents_intensive_capstone.agents import blue_hat_agent, green_hat_agent, yellow_hat_agent

c:\Praveen\Personal\git\agents-intensive-capstone-2025\.venv\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.5) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


## Configure Retry Options¶

In [2]:
load_dotenv()

# 2. Access the variable
api_key = os.getenv("GOOGLE_API_KEY")
print("✅ Gemini API key setup complete.")

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

✅ Gemini API key setup complete.


In [3]:
model = Gemini(
    model="gemini-2.5-flash-lite",
    retry_options=retry_config,
)

In [4]:
blue_hat = blue_hat_agent.BlueHatAgent.create(
    model=model,
    prompt_folder=Path("../src/agents_intensive_capstone/prompts"),
)


In [6]:
# GREEN HAT: Creativity & Alternatives
green_hat = green_hat_agent.GreenHatAgent.create(
    model=model,
    prompt_folder=Path("../src/agents_intensive_capstone/prompts"),
)

In [7]:
# YELLOW HAT: Optimism & Benefits
yellow_hat = yellow_hat_agent.YellowHatAgent.create(
    model=model,
    prompt_folder=Path("../src/agents_intensive_capstone/prompts"),
)

## Six thinking hats

In [8]:
# Define the 5 "Thinking" Agents (The Team)
# WHITE HAT: Facts & Data
white_hat = Agent(
    name="WhiteHat",
    model=model,
    instruction="You are the White Hat. Focus ONLY on available data, facts, and information gaps. Be objective and neutral. Do not offer opinions, only verifiable facts or questions about missing data.",
)

# RED HAT: Emotions & Intuition
red_hat = Agent(
    name="RedHat",
    model=model,
    instruction="You are the Red Hat. Focus on intuition, hunches, and emotional reaction. How does this problem make users or the team feel? You do not need to justify your feelings with logic.",
)

# BLACK HAT: Caution & Risk
black_hat = Agent(
    name="BlackHat",
    model=model,
    instruction="You are the Black Hat. Play the devil's advocate. Identify specific risks, potential failure points, downsides, and why this idea might NOT work. Be critical.",
)

In [9]:
# Create the Parallel Group
# This runs all 5 agents at the same time on the user prompt
thinking_team = ParallelAgent(
    name="SixHatsBrainstorm",
    sub_agents=[white_hat, red_hat, black_hat, yellow_hat, green_hat]
)

In [10]:
# Create the Final Workflow
# First run the team (Parallel), then run the manager (Sequential)
solver_workflow = SequentialAgent(
    name="SixHatsSolver",
    sub_agents=[thinking_team, blue_hat]
)

In [11]:
from google.adk.runners import InMemoryRunner
from google.adk.plugins.logging_plugin import (
    LoggingPlugin,
)  # <---- 1. Import the Plugin
from google.genai import types
import asyncio

runner = InMemoryRunner(
    agent=solver_workflow,
    plugins=[
        LoggingPlugin()
    ], 
)
print("✅ Runner configured")

✅ Runner configured


In [12]:
response = await runner.run_debug("Should we switch our backend database from PostgreSQL to a NoSQL solution for our startup?")


 ### Created new session: debug_session_id

User > Should we switch our backend database from PostgreSQL to a NoSQL solution for our startup?
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-f0d111e9-9aab-4684-9d6e-82bb537d1c99
[logging_plugin]    Session ID: debug_session_id
[logging_plugin]    User ID: debug_user_id
[logging_plugin]    App Name: InMemoryRunner
[logging_plugin]    Root Agent: SixHatsSolver
[logging_plugin]    User Content: text: 'Should we switch our backend database from PostgreSQL to a NoSQL solution for our startup?'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-f0d111e9-9aab-4684-9d6e-82bb537d1c99
[logging_plugin]    Starting Agent: SixHatsSolver
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: SixHatsSolver
[logging_plugin]    Invocation ID: e-f0d111e9-9aab-4684-9d6e-82bb537d1c99
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: SixHatsBrainstorm
[logging_plugin]    Invoc

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: YellowHatAgent
[logging_plugin]    Content: function_call: googleAgent
[logging_plugin]    Token Usage - Input: 308, Output: 28
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 5a213521-a823-4dd9-a107-52714868ac94
[logging_plugin]    Author: YellowHatAgent
[logging_plugin]    Content: function_call: googleAgent
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['googleAgent']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: googleAgent
[logging_plugin]    Agent: YellowHatAgent
[logging_plugin]    Function Call ID: adk-c5231236-46f4-44cc-9179-a96d531e4bdc
[logging_plugin]    Arguments: {'request': 'Pros and cons of switching from PostgreSQL to a NoSQL database for a startup'}
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-050a7286-1f8b-4642-9255-754ecab51310
[logging_plugin]    Session ID: 2e72bd80-6c4c-4b12-b5c3-7b7fe1cc58bd
[logging_plugin]